<a href="https://colab.research.google.com/github/CHL-edu/undergradutate/blob/main/SimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from pathlib import Path
import seaborn as sns
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# 常量定义
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
CHANNELS = 3
DATASET_SIZE = 1000

# 定义路径常量
LOAD_H5_PATH = Path('/content/drive/MyDrive/Colab Notebooks/person_train_394_White.h5')
MODEL_SAVE_PATH = Path('/content/drive/MyDrive/Colab Notebooks/output')
PROCESSED_OUTPUT_PATH = Path('/content/drive/MyDrive/Colab Notebooks/output')

# 确保输出目录存在
PROCESSED_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
MODEL_SAVE_PATH.mkdir(parents=True, exist_ok=True)


def create_improved_cnn(input_shape, output_size):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(output_size)
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model


def setup_callbacks():
    return [
        EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True),
        ModelCheckpoint(
            filepath =str(MODEL_SAVE_PATH / 'best_model.keras'),
            monitor='val_loss',                                             # 监控指标
            verbose=1,                                                      # 打印日志信息
            save_best_only=True,                                            # 只保留最优模型
            save_weights_only=False,                                        # 是否仅保存权重而非整个模型
            mode='auto',                                                     # 当监控指标越低越好时设为'min'
            save_freq='epoch',                                              # 每轮结束后保存一次
            initial_value_threshold=None
        ),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, mode='min', verbose=1)
    ]


def load_data(hdf5_path):
    with h5py.File(hdf5_path, 'r') as f:
        images = np.array(f['images'])
        keypoints = np.array(f['keypoints'])[:, :, :2]
    # 确保图像数据在 [0, 1] 范围内
    images = images.astype('float32')
    images = np.clip(images, 0, 1)  # 裁剪到 [0, 1]
    keypoints = keypoints.astype('float32')
    return images, keypoints


def split_data(images, keypoints):
    return train_test_split(images, keypoints, test_size=0.2, random_state=42, shuffle=True)


def plot_training_history(history, save_path):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title('Training vs Validation MAE')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


def plot_keypoint_comparison(images, true_kps, pred_kps, indices, save_path):
    n_samples = len(indices)
    plt.figure(figsize=(5 * n_samples, 10))
    for i, idx in enumerate(indices):
        # 确保图像数据在 [0, 1] 范围内
        img = images[idx]
        img = np.clip(img, 0, 1)  # 再次裁剪以确保安全

        # 真实关键点
        plt.subplot(2, n_samples, i + 1)
        plt.imshow(img)
        plt.scatter(true_kps[idx, :, 0] * IMAGE_WIDTH, true_kps[idx, :, 1] * IMAGE_HEIGHT,
                    c='red', label='True', s=10)
        plt.title(f'Sample {idx} - True')
        plt.axis('off')

        # 预测关键点
        plt.subplot(2, n_samples, i + 1 + n_samples)
        plt.imshow(img)
        pred_kps_reshaped = pred_kps[idx].reshape(-1, 2)
        plt.scatter(pred_kps_reshaped[:, 0] * IMAGE_WIDTH, pred_kps_reshaped[:, 1] * IMAGE_HEIGHT,
                    c='blue', label='Predicted', s=10)
        plt.title(f'Sample {idx} - Predicted')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


def plot_error_distribution(true_kps, pred_kps, save_path):
    errors = np.abs(true_kps.reshape(true_kps.shape[0], -1, 2) -
                    pred_kps.reshape(pred_kps.shape[0], -1, 2))
    errors = np.sqrt(np.sum(errors ** 2, axis=2))

    plt.figure(figsize=(10, 6))
    for i in range(errors.shape[1]):
        sns.kdeplot(errors[:, i], label=f'Keypoint {i + 1}')
    plt.title('Keypoint Prediction Error Distribution')
    plt.xlabel('Euclidean Error (pixels)')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()


def plot_error_heatmap(true_kps, pred_kps, save_path):
    errors = np.abs(true_kps.reshape(true_kps.shape[0], -1, 2) -
                    pred_kps.reshape(pred_kps.shape[0], -1, 2))
    errors = np.sqrt(np.sum(errors ** 2, axis=2))

    plt.figure(figsize=(12, 8))
    sns.heatmap(errors.T, cmap='viridis', cbar_kws={'label': 'Error (pixels)'})
    plt.title('Keypoint Prediction Error Heatmap')
    plt.xlabel('Sample Index')
    plt.ylabel('Keypoint Index')
    plt.savefig(save_path)
    plt.close()


def main():
    # 加载数据
    hdf5_path = LOAD_H5_PATH
    images, keypoints = load_data(hdf5_path)

    # 检查数据范围（用于调试）
    print(f"Image data range: min={images.min():.4f}, max={images.max():.4f}")

    # 分割数据
    X_train, X_val, y_train, y_val = split_data(images, keypoints)

    # 创建模型
    output_size = y_train.shape[1] * y_train.shape[2]
    model = create_improved_cnn((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS), output_size)
    model.summary()

    # 训练模型
    history = model.fit(
        X_train,
        y_train.reshape(y_train.shape[0], -1),
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        callbacks=setup_callbacks(),
        verbose=1
    )

    # 评估模型
    val_loss, val_mae = model.evaluate(X_val, y_val.reshape(y_val.shape[0], -1))
    print(f'Validation Loss: {val_loss:.4f}, Validation MAE: {val_mae:.4f}')

    # 保存模型
    model_save_path = MODEL_SAVE_PATH / f'person_model_{DATASET_SIZE}.keras'
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # 绘制训练历史
    history_plot_path = PROCESSED_OUTPUT_PATH / f'training_history_{DATASET_SIZE}.png'
    plot_training_history(history, history_plot_path)

    # 预测
    predictions = model.predict(X_val)

    # 绘制多样本关键点对比（显示4个样本）
    indices = [0, 10, 20, 30]
    kp_comparison_path = PROCESSED_OUTPUT_PATH / f'keypoint_comparison_{DATASET_SIZE}.png'
    plot_keypoint_comparison(X_val, y_val, predictions, indices, kp_comparison_path)

    # 绘制误差分布
    error_dist_path = PROCESSED_OUTPUT_PATH / f'error_distribution_{DATASET_SIZE}.png'
    plot_error_distribution(y_val, predictions, error_dist_path)

    # 绘制误差热图
    error_heatmap_path = PROCESSED_OUTPUT_PATH / f'error_heatmap_{DATASET_SIZE}.png'
    plot_error_heatmap(y_val, predictions, error_heatmap_path)


if __name__ == '__main__':
    main()

In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import drive
import os
import pandas as pd
import shutil

# 挂载 Google Drive 到 Colab 环境
drive.mount('/content/drive')

# 定义路径配置
CONFIG = {
    'base_path': '/content/drive/MyDrive/Colab Notebooks',
    'input_file': 'data/person_train_1000_White.h5',
    'output_dir': 'RESoutput',
    'subdirs': ['processoutput', 'output_Processed']
}

# 创建路径
SAVE_PATH = os.path.join(CONFIG['base_path'], CONFIG['output_dir'])
INPUT_PATH = os.path.join(CONFIG['base_path'], CONFIG['input_file'])

# 清空 output_Processed 目录
output_processed_path = os.path.join(SAVE_PATH, 'output_Processed')
if os.path.exists(output_processed_path):
    shutil.rmtree(output_processed_path)
    print(f"已清空目录: {output_processed_path}")
os.makedirs(output_processed_path, exist_ok=True)

# 创建或保留其他子目录
for subdir in CONFIG['subdirs']:
    if subdir != 'output_Processed':  # 已处理 output_Processed
        os.makedirs(os.path.join(SAVE_PATH, subdir), exist_ok=True)

# 图像尺寸
resize_width = 256
resize_height = 256

def create_model(input_shape, output_size):
    """创建 ResNet50 模型用于关键点预测"""
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False  # 冻结预训练层
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_size, activation='linear')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def load_and_prepare_data(hdf5_path):
    """加载并准备数据"""
    with h5py.File(hdf5_path, 'r') as f:
        images = np.array(f['images'])
        keypoints = np.array(f['keypoints'])[:, :, :2]  # 提取 x, y 坐标
    X_train, X_val, y_train, y_val = train_test_split(images, keypoints, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def plot_predictions(images, keypoints, predictions, indices, save_path):
    """绘制预测与真实关键点的对比图"""
    for idx in indices:
        plt.figure(figsize=(10, 5))
        # 真实关键点
        plt.subplot(1, 2, 1)
        plt.imshow(images[idx])
        plt.scatter(keypoints[idx, :, 0], keypoints[idx, :, 1], c='red', s=20, label='Ground Truth')
        plt.title('Ground Truth')
        plt.legend()
        plt.axis('off')
        # 预测关键点
        plt.subplot(1, 2, 2)
        plt.imshow(images[idx])
        pred_keypoints = predictions[idx].reshape(-1, 2)
        plt.scatter(pred_keypoints[:, 0], pred_keypoints[:, 1], c='blue', s=20, label='Predictions')
        plt.title('Predictions')
        plt.legend()
        plt.axis('off')
        plt.tight_layout()
        # 保存对比图
        plt.savefig(os.path.join(save_path, f'prediction_comparison_{idx}.png'), dpi=300)
        plt.close()

def plot_error_analysis(keypoints, predictions, save_path):
    """绘制关键点预测的误差分析图"""
    errors = np.abs(keypoints - predictions).reshape(-1, 2)
    plt.figure(figsize=(8, 6))
    plt.boxplot([errors[:, 0], errors[:, 1]], labels=['X Error', 'Y Error'])
    plt.title('Keypoint Prediction Error Distribution')
    plt.ylabel('Absolute Error (pixels)')
    plt.grid(True)
    plt.savefig(os.path.join(save_path, 'error_analysis.png'), dpi=300)
    plt.close()

def save_training_log(history, save_path):
    """保存训练日志为 CSV 文件"""
    log_df = pd.DataFrame({
        'epoch': range(1, len(history.history['loss']) + 1),
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss'],
        'train_mae': history.history['mae'],
        'val_mae': history.history['val_mae']
    })
    log_df.to_csv(os.path.join(save_path, 'training_log.csv'), index=False)

if __name__ == '__main__':
    # 加载数据
    X_train, X_val, y_train, y_val = load_and_prepare_data(INPUT_PATH)

    # 归一化数据
    X_train = X_train.astype('float32') / 255.0
    X_val = X_val.astype('float32') / 255.0
    y_train = y_train.astype('float32') / np.array([resize_width, resize_height])
    y_val = y_val.astype('float32') / np.array([resize_width, resize_height])

    # 创建模型
    output_size = y_train.shape[1] * y_train.shape[2]  # 关键点数量 * 2 (x, y)
    model = create_model((resize_width, resize_height, 3), output_size)
    model.summary()

    # 定义回调
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)

    # 训练模型
    history = model.fit(
        X_train,
        y_train.reshape(y_train.shape[0], -1),
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        callbacks=[early_stopping, reduce_lr]
    )

    # 评估模型
    val_loss, val_mae = model.evaluate(X_val, y_val.reshape(y_val.shape[0], -1))
    print(f'验证损失: {val_loss:.4f}, 验证 MAE: {val_mae:.4f}')

    # 预测并还原坐标
    predictions = model.predict(X_val)
    predictions = predictions.reshape(-1, y_val.shape[1], y_val.shape[2]) * np.array([resize_width, resize_height])
    y_val_orig = y_val * np.array([resize_width, resize_height])
    X_val_orig = X_val * 255.0  # 还原图像像素值以便可视化

    # 可视化多个样本的预测结果
    sample_indices = np.random.choice(X_val.shape[0], 5, replace=False)
    plot_predictions(X_val_orig, y_val_orig, predictions, sample_indices, os.path.join(SAVE_PATH, 'output_Processed'))

    # 误差分析
    plot_error_analysis(y_val_orig, predictions, os.path.join(SAVE_PATH, 'output_Processed'))

    # 保存训练日志
    save_training_log(history, os.path.join(SAVE_PATH, 'output_Processed'))

    # 绘制并保存训练指标图
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='训练损失')
    plt.plot(history.history['val_loss'], label='验证损失')
    plt.title('训练与验证损失')
    plt.xlabel('轮次')
    plt.ylabel('损失')
    plt.legend()
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='训练 MAE')
    plt.plot(history.history['val_mae'], label='验证 MAE')
    plt.title('训练与验证 MAE')
    plt.xlabel('轮次')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, 'output_Processed', 'loss_mae_TrainRes_1000.png'), dpi=300)
    plt.close()

In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import drive
import os
import pandas as pd
import shutil
import timm

# 挂载 Google Drive 到 Colab 环境
drive.mount('/content/drive')

# 定义路径配置
CONFIG = {
    'base_path': '/content/drive/MyDrive/Colab Notebooks',
    'input_file': 'data/person_train_1000_White.h5',
    'output_dir': 'RESoutput',
    'subdirs': ['processoutput', 'output_Processed'],
    'model_weights': 'models/pose_hrnet_w32_256x192.h5'  # 假设转换后的 TensorFlow 权重路径
}

# 创建路径
SAVE_PATH = os.path.join(CONFIG['base_path'], CONFIG['output_dir'])
INPUT_PATH = os.path.join(CONFIG['base_path'], CONFIG['input_file'])
WEIGHTS_PATH = os.path.join(CONFIG['base_path'], CONFIG['model_weights'])

# 清空 output_Processed 目录
output_processed_path = os.path.join(SAVE_PATH, 'output_Processed')
if os.path.exists(output_processed_path):
    shutil.rmtree(output_processed_path)
    print(f"已清空目录: {output_processed_path}")
os.makedirs(output_processed_path, exist_ok=True)

# 创建或保留其他子目录
for subdir in CONFIG['subdirs']:
    if subdir != 'output_Processed':
        os.makedirs(os.path.join(SAVE_PATH, subdir), exist_ok=True)

# 图像尺寸
resize_width = 256
resize_height = 256

def create_model(input_shape, output_size):
    """创建 HRNet-W32 模型，加载 COCO 预训练权重"""
    # 使用 timm 加载 HRNet-W32 骨干（ImageNet 预训练）
    base_model = timm.create_model('hrnet_w32', pretrained=True, num_classes=0)

    # 获取骨干网络输出
    x = base_model.forward_features(tf.keras.Input(shape=input_shape))
    x = GlobalAveragePooling2D()(x)

    # 添加全连接层
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_size, activation='linear')(x)

    # 创建模型
    model = Model(inputs=base_model.input, outputs=predictions)

    # 尝试加载 COCO 预训练权重（假设已转换为 .h5 格式）
    if os.path.exists(WEIGHTS_PATH):
        model.load_weights(WEIGHTS_PATH)
        print(f"已加载 COCO 预训练权重: {WEIGHTS_PATH}")
    else:
        print(f"警告: 未找到 COCO 预训练权重 {WEIGHTS_PATH}，使用 ImageNet 预训练权重")

    # 冻结骨干网络
    for layer in base_model.layers:
        layer.trainable = False

    # 编译模型
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def load_and_prepare_data(hdf5_path):
    """加载并准备数据"""
    with h5py.File(hdf5_path, 'r') as f:
        images = np.array(f['images'])
        keypoints = np.array(f['keypoints'])[:, :, :2]
    X_train, X_val, y_train, y_val = train_test_split(images, keypoints, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def plot_predictions(images, keypoints, predictions, indices, save_path):
    """绘制预测与真实关键点的对比图"""
    for idx in indices:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(images[idx])
        plt.scatter(keypoints[idx, :, 0], keypoints[idx, :, 1], c='red', s=20, label='Ground Truth')
        plt.title('Ground Truth')
        plt.legend()
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(images[idx])
        pred_keypoints = predictions[idx].reshape(-1, 2)
        plt.scatter(pred_keypoints[:, 0], pred_keypoints[:, 1], c='blue', s=20, label='Predictions')
        plt.title('Predictions')
        plt.legend()
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f'prediction_comparison_{idx}.png'), dpi=300)
        plt.close()

def plot_error_analysis(keypoints, predictions, save_path):
    """绘制关键点预测的误差分析图"""
    errors = np.abs(keypoints - predictions).reshape(-1, 2)
    plt.figure(figsize=(8, 6))
    plt.boxplot([errors[:, 0], errors[:, 1]], labels=['X Error', 'Y Error'])
    plt.title('Keypoint Prediction Error Distribution')
    plt.ylabel('Absolute Error (pixels)')
    plt.grid(True)
    plt.savefig(os.path.join(save_path, 'error_analysis.png'), dpi=300)
    plt.close()

def save_training_log(history, save_path):
    """保存训练日志为 CSV 文件"""
    log_df = pd.DataFrame({
        'epoch': range(1, len(history.history['loss']) + 1),
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss'],
        'train_mae': history.history['mae'],
        'val_mae': history.history['val_mae']
    })
    log_df.to_csv(os.path.join(save_path, 'training_log.csv'), index=False)

if __name__ == '__main__':
    # 加载数据
    X_train, X_val, y_train, y_val = load_and_prepare_data(INPUT_PATH)

    # 归一化数据
    X_train = X_train.astype('float32') / 255.0
    X_val = X_val.astype('float32') / 255.0
    y_train = y_train.astype('float32') / np.array([resize_width, resize_height])
    y_val = y_val.astype('float32') / np.array([resize_width, resize_height])

    # 创建模型
    output_size = y_train.shape[1] * y_train.shape[2]
    model = create_model((resize_width, resize_height, 3), output_size)
    model.summary()

    # 定义回调
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)

    # 训练模型
    history = model.fit(
        X_train,
        y_train.reshape(y_train.shape[0], -1),
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        callbacks=[early_stopping, reduce_lr]
    )

    # 评估模型
    val_loss, val_mae = model.evaluate(X_val, y_val.reshape(y_val.shape[0], -1))
    print(f'验证损失: {val_loss:.4f}, 验证 MAE: {val_mae:.4f}')

    # 预测并还原坐标
    predictions = model.predict(X_val)
    predictions = predictions.reshape(-1, y_val.shape[1], y_val.shape[2]) * np.array([resize_width, resize_height])
    y_val_orig = y_val * np.array([resize_width, resize_height])
    X_val_orig = X_val * 255.0

    # 可视化多个样本的预测结果
    sample_indices = np.random.choice(X_val.shape[0], 5, replace=False)
    plot_predictions(X_val_orig, y_val_orig, predictions, sample_indices, output_processed_path)

    # 误差分析
    plot_error_analysis(y_val_orig, predictions, output_processed_path)

    # 保存训练日志
    save_training_log(history, output_processed_path)

    # 绘制并保存训练指标图
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='训练损失')
    plt.plot(history.history['val_loss'], label='验证损失')
    plt.title('训练与验证损失')
    plt.xlabel('轮次')
    plt.ylabel('损失')
    plt.legend()
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='训练 MAE')
    plt.plot(history.history['val_mae'], label='验证 MAE')
    plt.title('训练与验证 MAE')
    plt.xlabel('轮次')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_processed_path, 'loss_mae_TrainHRNet_COCO_1000.png'), dpi=300)
    plt.close()